In [ ]:
lvl           = None
Density_uod_l = None
file_path_Sgm = None
file_path_Lvl = None

In [ ]:
%run ./4___UOD_vals_and_lvls___Functions.ipynb

In [ ]:
Density_uod_l = np.array(eval(Density_uod_l))

---
---
---

In [ ]:
unique_Density_ud_l = sorted(set(interval[0] for interval in Density_uod_l))
unique_Density_od_l = sorted(set(interval[1] for interval in Density_uod_l))

### CDF fit

In [ ]:
with open(file_path_Sgm+"grid_fft.pk",  'rb') as f: grid_fft = pkl.load(f)

In [ ]:
data = np.log10(grid_fft).flatten()
x_vals = np.linspace(data.min(), data.max(), 10**4)

In [ ]:
del grid_fft; gc.collect()

In [ ]:
density_vals = fastKDE_pdf(data)

x_grid = np.linspace(data.min(), data.max(), len(density_vals))

density_vals_interp = np.interp(x_vals, x_grid, density_vals)
density_vals_interp /= np.sum(density_vals_interp)

In [ ]:
cdf_vals = cumulative_trapezoid(density_vals_interp, x_vals, initial=0)
cdf_vals /= cdf_vals[-1]

In [ ]:
del data, density_vals, density_vals_interp; gc.collect()

## Values (FFT)

In [ ]:
ud_vals = []; od_vals = []

i0 = 0
for i, ii in enumerate(cdf_vals):
    if ii >= unique_Density_ud_l[i0]:
        ud_vals.append(mfloor(10**x_vals[i]*10**3)/10**3)
        i0 += 1
    if i0 == len(unique_Density_ud_l): break

i0 = 0
for i, ii in enumerate(cdf_vals[::-1]):
    if ii <= unique_Density_od_l[::-1][i0]:
        od_vals.append(mceil(10**x_vals[::-1][i]*10**3)/10**3)
        i0 += 1
    if i0 == len(unique_Density_od_l): break

## Levels

In [ ]:
with open(file_path_Sgm+"grid_fft.pk",  'rb') as f: grid_fft  = pkl.load(f)
with open(file_path_Lvl+"grid_lvld.pk", 'rb') as f: grid_lvld = pkl.load(f)

In [ ]:
ud_lvls = []
grid_ud = np.full(grid_lvld.shape, False)
ud_lvl = 0

for ud_val in ud_vals:
    grid_ud[grid_fft <= ud_val] = True
    if np.max(grid_ud): ud_lvl = np.max(grid_lvld[grid_ud])
    ud_lvls.append(ud_lvl)

In [ ]:
del grid_ud; gc.collect()

In [ ]:
od_lvls = []
grid_od = np.full(grid_lvld.shape, False)
od_lvl = lvl

for od_val in od_vals:
    grid_od[od_val <= grid_fft] = True
    if np.max(grid_od): od_lvl = np.min(grid_lvld[grid_od])
    od_lvls.append(od_lvl)

In [ ]:
del grid_od; gc.collect()

## Save the Data

In [ ]:
for d,Density_ud in enumerate(unique_Density_ud_l):
    with open(file_path_Lvl+"ud_val___"+str(Density_ud)+".pk", 'wb') as f: pkl.dump(ud_vals[d], f)
    with open(file_path_Lvl+"ud_lvl___"+str(Density_ud)+".pk", 'wb') as f: pkl.dump(ud_lvls[d], f)

for d,Density_od in enumerate(unique_Density_od_l[::-1]):
    with open(file_path_Lvl+"od_val___"+str(Density_od)+".pk", 'wb') as f: pkl.dump(od_vals[d], f)
    with open(file_path_Lvl+"od_lvl___"+str(Density_od)+".pk", 'wb') as f: pkl.dump(od_lvls[d], f)

The idea is that now we can check which levels are below that threshold and just sort all of their elements and not ned to look for pair origins outside of that range (which takes a lot of time) nor, in the next notebooks, look for pairs within remaining levels (also a huge amount of computaitonal time).

---
---
---